In [67]:
import os.path
import sqlite3
import cohere
import numpy as np

In [68]:
def give_list(db, name=''):
	if len(name) != 0:
		#print(name + ':')
		res_table = db.execute(f'SELECT * FROM {name}')
		rows = [row[0] for row in res_table]
		return rows
	else:
		rows = [row[0] for row in db]
		return rows

In [69]:
title = "bruh, and muh v. bruh, sdfsd"
', '.join(title.split(', ')[:-1]).split('(')[0].rstrip()

'bruh, and muh v. bruh'

In [70]:
con = sqlite3.connect('courtCases_old.db')
cur = con.cursor()

In [71]:
command = """
SELECT name
FROM Cases C
"""

res_table = cur.execute(command)
names = give_list(res_table)

command = """
SELECT content
FROM Cases C
"""

res_table = cur.execute(command)
content = give_list(res_table)

docs = [n + ': ' + c for n in names for c in content]

In [72]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [73]:
cohere_key = os.getenv("COHERE_API_KEY")
co = cohere.Client(cohere_key)

In [74]:
doc_emb = co.embed(docs, input_type="search_document", model="embed-english-v3.0").embeddings
doc_emb = np.asarray(doc_emb)
doc_emb

KeyboardInterrupt: 

In [ ]:
doc_emb.shape

(200, 1024)

In [ ]:
#Encode your query with input type 'search_query'
query = "Is there any precedent for calling a second-hand witness to the stand?"
query_emb = co.embed([query], input_type="search_query", model="embed-english-v3.0").embeddings
query_emb = np.asarray(query_emb)
query_emb.shape

(1, 1024)

In [ ]:
#Compute the dot product between query embedding and document embedding
scores = np.dot(query_emb, doc_emb.T)[0]

#Find the highest scores
max_idx = np.argsort(-scores)

count = 0
print(f"Query: {query}")
for idx in max_idx:
  print(f"Score: {scores[idx]:.2f}")
  print(docs[idx])
  print("--------")
  count += 1
  if count == 3:
    break;

Query: Is there any precedent for calling a second-hand witness to the stand?
Score: 0.43
ORDER

This case is restored to the calendar for reargument. Counsel are requested to brief and discuss during oral argument the following questions: [434 U.S. 980 , 981]   1. Is the rule heretofore applied in the federal courts-that jeopardy attaches in jury trials when the jury is sworn-constitutionally mandated?

2. Should this Court hold that the Constitution does not require jeopardy to attach in any trial-state or federal, jury or nonjury- until the first witness is sworn?

The Solicitor General is invited to file a brief expressing th views of the United States on each of these questions.

Mr. Justice MARSHALL, dissenting.

By its order restoring this case to the calendar for rebriefing and additional oral argument, the Court appears once again to be "reach[ing] out" for a vehicle to change a long line of precedent. See Pennsylvania v. Mimms, 434 U.S. 106 , at 117, (STEVENS, J., dissenting)

In [ ]:
def cosine_sim(vector, matrix):
    scores = []
    for mat in matrix:
        cos_sim = np.dot(vector, mat)/(np.linalg.norm(vector)*np.linalg.norm(mat))
        scores.append(cos_sim[0])
    return np.asarray(scores)

#Compute the cosine similarity between query embedding and document embedding
scores = cosine_sim(query_emb, doc_emb)

#Find the highest scores
max_idx = np.argsort(-scores)

count = 0
print(f"Query: {query}")
for idx in max_idx:
  print(f"Score: {scores[idx]:.2f}")
  print(docs[idx])
  print("--------")
  count += 1
  if count == 3:
    break;

Query: Is there any precedent for calling a second-hand witness to the stand?
Score: 0.43
ORDER

This case is restored to the calendar for reargument. Counsel are requested to brief and discuss during oral argument the following questions: [434 U.S. 980 , 981]   1. Is the rule heretofore applied in the federal courts-that jeopardy attaches in jury trials when the jury is sworn-constitutionally mandated?

2. Should this Court hold that the Constitution does not require jeopardy to attach in any trial-state or federal, jury or nonjury- until the first witness is sworn?

The Solicitor General is invited to file a brief expressing th views of the United States on each of these questions.

Mr. Justice MARSHALL, dissenting.

By its order restoring this case to the calendar for rebriefing and additional oral argument, the Court appears once again to be "reach[ing] out" for a vehicle to change a long line of precedent. See Pennsylvania v. Mimms, 434 U.S. 106 , at 117, (STEVENS, J., dissenting)

In [ ]:
max_idx

array([ 41,  81, 121,  24,  42,  39, 108, 185,   0, 183, 180, 192, 138,
        97, 134, 156, 147, 143, 124, 193,  37, 190, 118,  78,  85,  32,
        93, 140, 176, 197,  71, 125,  51, 177, 105, 172,  83,  61,  45,
       189,  82, 166,  30, 116,  87,  21, 165,  62,  57, 155, 102, 195,
        27,  88, 184,  77,  34, 128,  16,  70,  92, 168,  48, 162,  56,
       126,  18, 144, 119, 153,  75, 114,  55,  14,  67, 194, 109,  40,
        19, 170,  66,   2,  69, 132,  46,  25,  26,   9, 163,  52, 171,
       149, 182,  65, 142,  99,  98, 164, 161,  28, 148, 107,  22, 187,
        20, 113, 154,   1,  59,  13,  17, 103, 141, 199, 111,  11,  53,
       198, 178, 158, 137,   6,  96,  90,  54,  86,  94, 181, 104, 127,
       167,  79,  33, 152, 122,   3, 100, 196, 186,   5,  12,  63, 130,
         4,  72,  15,   7, 173, 151, 106, 117,  68,  31, 169,  23,  64,
        89, 120, 101,  60, 150, 135, 131, 188, 139,  49, 129,  84, 159,
        74, 179,  73, 191,  58,  50, 174,  76, 133, 146,  36,  3

In [ ]:
results = co.rerank(query=query, documents=[docs[idx] for idx in max_idx], top_n=3, model="rerank-english-v2.0")

print(f"Query: {query}")
for res in results:
  print(res.document['text'])
  print("--------")

Query: Is there any precedent for calling a second-hand witness to the stand?
US v. PACIFIC R R
--------
IN RE EASTERN CHEROKEES
--------
US v. TERMINAL RAILROAD ASS'N OF ST LOUIS
--------


In [ ]:
for i in range(len(docs)):
    if docs[i][:20].lower == results[0].document['text'][:20]:
        print(i)
        break;